# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

In [2]:
import os
from huggingface_hub import login

login(new_session=False, token=os.getenv("HF_TOKEN"))

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="auto")

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find the main Batman The Dark Knight filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [6]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [7]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the main Batman The Dark Knight filming locations in the world, calculate the time to transfer via cargo   │
│ plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.           │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Find main filming locations for "The Dark Knight"                                                              
  batman_locations = web_search("main filming locations The Dark Knight")                                          
  print("Batman The Dark Knight filming locations:", batman_locations)                                             
                                                                                                                   
  # Coordinates for Gotham (New York City)                                                                         
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Coordinates for Chicago                                                                                        
  chicago_coords = (41.8781, -87.6298)                                                                             
                                                                                                                   
  # Calculate travel time from Chicago to Gotham                                                                   
  travel_time_chicago = calculate_cargo_travel_time(origin_coords=chicago_coords,                                  
  destination_coords=gotham_coords)                                                                                
  print("Travel time from Chicago to Gotham:", travel_time_chicago)                                                
                                                                                                                   
  # Coordinates for New York City                                                                                  
  nyc_coords = (40.7128, -74.0060)                                                                                 
                                                                                                                   
  # Calculate travel time from New York City to Gotham                                                             
  travel_time_nyc = calculate_cargo_travel_time(origin_coords=nyc_coords, destination_coords=gotham_coords)        
  print("Travel time from New York City to Gotham:", travel_time_nyc)                                              
                                                                                                                   
  # Find some supercar factories                                                                                   
  supercar_factories = web_search("supercar factories locations")                                                  
  print("Supercar factories locations:", supercar_factories)                                                       
                                                                                                                   
  import pandas as pd                                                                                              
                                                                                                                   
  # Coordinates for supercar factories                                                                             
  supercar_coords = {                                                                                              
      "Lamborghini - Sant'Agata Bolognese, Italy": (44.5055, 11.3364),                                             
      "Ferrari - Maranello, Italy": (44.5646, 10.9941),                                                            
      "Porsche - Leipzig, Germany": (51.3397, 12.3731),                                                            
      "Koenigsegg - Angered, Sweden": (59.3293, 18.0686),

Execution logs:
Batman The Dark Knight filming locations: ## Search Results

[Film Locations for Christopher Nolan's The Dark Knight, in Chicago 
...](https://www.movie-locations.com/movies/d/Dark-Knight.php)
The Dark Knight, Christopher Nolan's follow-up to Batman Begins, expands Batman's universe with plenty of real 
locations in Chicago, Hong Kong and London.Memories of Adam West and Cesar Romero 's Sixties hi-jinks are banished 
completely as the mood shifts from dark to downright grim.. As the tone changes, so 'Gotham' morphs from urban 
Gothic to sinister corporate sheen - and this ...

[The Dark Knight Filming Locations (2008)](https://a2zfilminglocation.com/the-dark-knight-filming-locations-2008/)
La La Land Filming Locations: The Dark Knight is an Action, Crime, Drama, and Thriller Film Which is directed by 
Christopher Nolan and Jonathan Nolan, Christopher Nolan, and David S. Goyer. Based on the DC Comics superhero 
Batman, the film is the 2nd installment of The Dark Knight trilogy and the sequel to the 2005 film Batman Begins.. 
The Film Stars include Christian Bale, Michael Caine ...

[The Dark Knight Filming Locations in Chicago, England, and Hong 
Kong](https://www.legendarytrips.com/the-dark-knight-filming-locations-2008/)
Not just an outstanding Batman movie, The Dark Knight is a masterpiece on its own that paves the way to, in my 
opinion, the best opus of Nolan's trilogy: The Dark Knight Rises. In The Dark Knight, Gotham City stays in Chicago 
with still a few filming locations in England. The main guest destination is Hong Kong where a few scenes were 
filmed ...

[Where Was The Dark Knight Filmed? Chicago & Other Filming 
Locations](https://thecinemaholic.com/where-did-the-shooting-of-the-dark-knight-take-place/)
The majority of the on set filming for 'The Dark Knight' was done at Pinewood Studios in Buckinghamshire, near 
London. Pinewood Studios is a popular filming location where numerous high profile movies have been filmed 
including 'Star Wars: Episode VIII - The Last Jedi', 'Jurassic World: Fallen Kingdom', 'Transformers: The Last ...

[Where was The Dark Knight filmed? - 
Giggster](https://giggster.com/guide/movie-location/where-was-the-dark-knight-filmed)
This The Dark Knight filming location is the impressive Criterion Restaurant attached to the Criterion Theater at 
224 Piccadilly in London, UK. This opulent restaurant was built in the Neo-Byzantine style by renowned architect 
Thomas Verity and was opened in 1873.

[The Dark Knight Locations - Movies Locations - 
LatLong](https://www.latlong.net/location/the-dark-knight-locations-118)
The Dark Knight was mainly filmed in Chicago, Illinois. The Sears Tower, Navy Pier, The Berghoff, the James R. 
Thompson Center, Trump International Hotel and Tower, Randolph Street Station, 330 North Wabash, LaSalle Street, 
and Hotel 71 were among the filming locations.

[The Dark Knight (2008) Film Locations](https://globalfilmlocations.net/2018/01/20/the-dark-knight-film-locations/)
Film locations for Christopher Nolan's 2008 Batman film The Dark Knight. Locations: 499-469 W Van Buren St, 
Chicago, IL 60607, USA (Bank Robbery) Google Maps Co-ordinates: 41.876782, -87.639278 427-499 N Franklin St, 
Chicago, IL 60654, USA (Lamborghini Crash) Google Maps Co-ordinates: 41.890693, -87.635550 Trump Tower, North 
Wabash Avenue, Chicago, IL, USA (Batman v The Joker) Google Maps…

[The Dark Knight (2008) | www.filming-locations.com](https://www.filming-locations.com/m2552-The-Dark-Knight)
T2 Trainspotting Taken Taken 2 Taxi Tea with Mussolini Ted Terminator 2: Judgment Day Terminator Genisys That Man 
from Rio The 15:17 to Paris The A-Team The Accountant The Affair of the Necklace The Amazing Spider-Man The 
Avengers The Beach The Best Offer The Big Lebowski The Big Runaround The Blind Side The Blue Lagoon The Bourne 
Identity The ...

[The Dark Knight - filming locations - SCEEN IT](https://www.sceen-it.com/movie/40/The-Dark-Knight)
All filming locations of The Dark Kn

[Step 1: Duration 15.61 seconds| Input tokens: 2,288 | Output tokens: 1,006]

In [8]:
result

,Location,Travel Time (hours)
0,"Chicago, Illinois",2.68
1,"New York City, New York",1.00
2,"Lamborghini - Sant'Agata Bolognese, Italy",10.77
3,"Ferrari - Maranello, Italy",10.73
4,"Porsche - Leipzig, Germany",10.36
5,"Koenigsegg - Angered, Sweden",10.27
6,"Pagani - San Cesario sul Panaro, Italy",10.74


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [9]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find the main Batman The Dark Knight filming locations in the world, calculate the time to transfer via cargo   │
│ plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.           │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We need to find main Batman The Dark Knight filming locations.
- We need to calculate the travel time for a cargo plane to Gotham for these locations.
- We need to find supercar factories with the same cargo plane transfer time to Gotham.

### 1.2. Facts to look up
- Main Batman The Dark Knight filming locations
  - Source: IMDB, Wikipedia, or other reliable movie production databases.
- Coordinates of these filming locations
  - Source: Google Maps, or the same movie production databases.
- Coordinates of supercar factories
  - Source: Manufacturer websites, automotive industry databases, or Wikipedia.

### 1.3. Facts to derive
- Travel time for a cargo plane from each filming location to Gotham
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of Gotham and each filming 
location.
- Travel time for a cargo plane from each supercar factory to Gotham
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of Gotham and each supercar 
factory.

## 2. Plan
1. Search for the main Batman The Dark Knight filming locations.
2. For each filming location, find its coordinates.
3. Search for the coordinates of supercar factories.
4. Calculate the travel time for a cargo plane from each filming location to Gotham.
5. Calculate the travel time for a cargo plane from each supercar factory to Gotham.
6. Create a pandas dataframe with the filming locations, their coordinates, and the calculated travel times.
7. Add supercar factories to the dataframe with their coordinates and the calculated travel times.
8. final_answer(answer)


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Search for the main Batman The Dark Knight filming locations                                           
  batman_locations_query = "Batman The Dark Knight filming locations"                                              
  batman_locations = web_search(query=batman_locations_query)                                                      
  print("Batman The Dark Knight filming locations:", batman_locations)                                             
                                                                                                                   
  # Step 2: Search for the coordinates of some supercar factories                                                  
  supercar_factories_query = "supercar factories coordinates"                                                      
  supercar_factories = web_search(query=supercar_factories_query)                                                  
  print("Supercar factories coordinates:", supercar_factories)                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman The Dark Knight filming locations: ## Search Results

[Film Locations for Christopher Nolan's The Dark Knight, in Chicago 
...](https://www.movie-locations.com/movies/d/Dark-Knight.php)
The Dark Knight, Christopher Nolan's follow-up to Batman Begins, expands Batman's universe with plenty of real 
locations in Chicago, Hong Kong and London.Memories of Adam West and Cesar Romero 's Sixties hi-jinks are banished 
completely as the mood shifts from dark to downright grim.. As the tone changes, so 'Gotham' morphs from urban 
Gothic to sinister corporate sheen - and this ...

[The Dark Knight Filming Locations (2008)](https://a2zfilminglocation.com/the-dark-knight-filming-locations-2008/)
La La Land Filming Locations: The Dark Knight is an Action, Crime, Drama, and Thriller Film Which is directed by 
Christopher Nolan and Jonathan Nolan, Christopher Nolan, and David S. Goyer. Based on the DC Comics superhero 
Batman, the film is the 2nd installment of The Dark Knight trilogy and the sequel to the 2005 film Batman Begins.. 
The Film Stars include Christian Bale, Michael Caine ...

[Where Was The Dark Knight Filmed? Chicago & Other Filming 
Locations](https://thecinemaholic.com/where-did-the-shooting-of-the-dark-knight-take-place/)
The city of Chicago was chosen to represent Gotham as the backdrop for 'The Dark Knight' by Nolan himself, who 
describes his experience filming there for 'Batman Begins' as "truly remarkable". Filming in Chicago took place in 
two phases, initially only for a few days in April, and from June to September 207 later on.

[The Dark Knight Filming Locations in Chicago, England, and Hong 
Kong](https://www.legendarytrips.com/the-dark-knight-filming-locations-2008/)
While keeping the dark tone of Batman Begins, The Dark Knight introduces a touch of modern minimalism with this new
filming location. The building is especially famous for its courtyard (Daley Plaza) featuring a 15-meter (50 ft) 
Cor-ten steel sculpture by Pablo Picasso, a gift from the artist to the city of Chicago in 1967.

[Filming Locations of Chicago and Los Angeles: The Dark 
Knight](https://www.itsfilmedthere.com/2012/11/the-dark-knight.html)
When Jokers location is being determined Batman is at about 300 east Wacker on the cycle heading then west towards 
Trump Tower. Ferries were filmed leaving Navy Pier. If remember other locations I will add it here in comments. I 
saw a lot of the filming of the movie in the Summer of 2007. Reply Delete

[Where was The Dark Knight filmed? - 
Giggster](https://giggster.com/guide/movie-location/where-was-the-dark-knight-filmed)
The Dark Knight was Christopher Nolan's second movie in the new Batman franchise series. It is the sequel to Batman
Begins and the second installment of The Dark Knight Trilogy, starring Christian Bale as Bruce Wayne/ Batman. The 
filming of The Dark Knight took place between April and November 2007 with a budget of $185 million.

[The Dark Knight (2008) - Filming & production - IMDb](https://m.imdb.com/title/tt0468569/locations/)
The Dark Knight: Directed by Christopher Nolan. With Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine. 
When a menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman, James Gordon and Harvey 
Dent must work together to put an end to the madness.

[The Dark Knight (2008) Film Locations](https://globalfilmlocations.net/2018/01/20/the-dark-knight-film-locations/)
Film locations for Christopher Nolan's 2008 Batman film The Dark Knight. Locations: 499-469 W Van Buren St, 
Chicago, IL 60607, USA (Bank Robbery) Google Maps Co-ordinates: 41.876782, -87.639278 427-499 N Franklin St, 
Chicago, IL 60654, USA (Lamborghini Crash) Google Maps Co-ordinates: 41.890693, -87.635550 Trump Tower, North 
Wabash Avenue, Chicago, IL, USA (Batman v The Joker) Google Maps…

[The Dark Knight - filming locations - SCEEN IT](https://www.sceen-it.com/movie/40/The-Dark-Knight)
Filming location of The Dark Knight (credits: Warner

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
  import pandas as pd                                                                                              
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Extract Batman The Dark Knight filming locations coordinates                                                   
  batman_locations_coords = []                                                                                     
  batman_locations_names = []                                                                                      
                                                                                                                   
  # Regex to find coordinates in the format (latitude, longitude)                                                  
  coords_regex = re.compile(r'\(([-+]?\d*\.\d+),\s*([-+]?\d*\.\d+)\)')                                             
                                                                                                                   
  for content in batman_locations_content:                                                                         
      matches = coords_regex.findall(content)                                                                      
      for match in matches:                                                                                        
          lat, lon = map(float, match)                                                                             
          batman_locations_coords.append((lat, lon))                                                               
          # Extract location name from the content                                                                 
          location_name = re.search(r'film location: (.+?):', content)                                             
          if location_name:                                                                                        
              batman_locations_names.append(location_name.group(1))                                                
          else:                                                                                                    
              batman_locations_names.append("Unknown")                                                             
                                                                                                                   
  # Extract Supercar factories coordinates                                                                         
  supercar_factories_coords = []                                                                                   
  supercar_factories_names = []                                                                                    
                                                                                                                   
  for content in supercar_factories_content:                                                                       
      matches = coords_regex.findall(content)                                                                      
      for match in matches:                                                                                        
          lat, lon = map(float, match)                                                                             
          supercar_factories_coords.append((lat, lon))   

Execution logs:
Empty DataFrame
Columns: [Location, Coordinates, Travel Time (hours)]
Index: []

Out - Final answer: Empty DataFrame
Columns: [Location, Coordinates, Travel Time (hours)]
Index: []

[Step 3: Duration 9.08 seconds| Input tokens: 31,079 | Output tokens: 1,529]

Empty DataFrame
Columns: [Location, Coordinates, Travel Time (hours)]
Index: []


In [10]:
detailed_report

,Location,Coordinates,Travel Time (hours)


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [11]:
model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="auto", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [12]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="auto", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly.*",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [13]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly.*', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │             

In [14]:
manager_agent.run("""
Find the main Batman The Dark Knight filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with similar cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the main Batman The Dark Knight filming locations in the world, calculate the time to transfer via cargo   │
│ plane to here (we're in Gotham, 40.7128° N, 74.0060° W).                                                        │
│ Also give me some supercar factories with similar cargo plane transfer time. You need at least 6 points in      │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ InferenceClientModel - deepseek-ai/DeepSeek-R1 ────────────────────────────────────────────────────────────────╯

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://router.huggingface.co/sambanova/v1/chat/completions (Request ID: Root=1-68576ec6-18af196b4a32db7f615573aa;e02a533f-3c69-41d2-99f6-1f0aa79b9983)

Something went wrong! Unknown error when running prediction service.

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)